## RESPUESTAS 
#### 1) Diferencias entre `QDA`y `TensorizedQDA`

1. ¿Sobre qué paraleliza `TensorizedQDA`? ¿Sobre las $k$ clases, las $n$ observaciones a predecir, o ambas?

Sobre las $k$ clases.

2. Analizar los shapes de `tensor_inv_covs` y `tensor_means` y explicar paso a paso cómo es que `TensorizedQDA` llega a predecir lo mismo que `QDA`.

`tensor_inv_covs` es un array de NumPy de $\mathbb{R}^{k \times p \times n}$

`tensor_means` es un array de NumPy de $\mathbb{R}^{k \times p \times 1}$

QDA en el método `_predict_one` itera sobre las $k$ clases calculando el logaritmo de la probabilidad a posteriori para cada una, usando el método `_predict_log_conditional` (este método devuelve un escalar), obteniendo una lista (de tamaño $k$) con la probabilidad de que la observación pertenezca a cada una de las clases. Luego, devuelve el argumento de la mayor probabilidad en dicha lista.

QDA
inv_cov # (p,p)
unbiased_x # (p,1)
return 0.5*np.log(LA.det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x
(1,p) @ (p,p) @ (p,1) → (1,1) → escalar.

TensorizedQDA directamente hace el cálculo de los logaritmos de las probabilidades a posteriori de forma matricial en el método `_predict_log_conditionals`, el cual directamente devuelve un vector columna $\mathbb{R}^{p \times 1}$, similar a la lista obtenida con el bucle for dentro del método `_predict_one` de la clase `QDA`. Luego, el método `_predict_one` para esta clase solo se encarga de encotrar el argumento que maximiza la probailidad (logarítmica).

tensor_inv_cov # 

### 2) Optimización

Debido a la forma cuadrática de QDA, no se puede predecir para $n$ observaciones en una sola pasada (utilizar $X \in \mathbb{R}^{p \times n}$ en vez de $x \in \mathbb{R}^p$) sin pasar por una matriz de $n \times n$ en donde se computan todas las interacciones entre observaciones. Se puede acceder al resultado recuperando sólo la diagonal de dicha matriz, pero resulta ineficiente en tiempo y (especialmente) en memoria. Aún así, es *posible* que el modelo funcione más rápido.

3. Implementar el modelo `FasterQDA` (se recomienda heredarlo de `TensorizedQDA`) de manera de eliminar el ciclo for en el método predict.

### Objetivo
Poder eliminar el ciclo for de predict. Es decir, predecir las n observaciones y k clases en un mismo paso. 

En BaseBayesianClassifier

predict(self, X) → tiene ciclo for recorre las n observaciones, y llama a predict_one, donde TensorizedQDA ya paraleliza las k clases. 

-------------
def predict(self, X): → llama a predict_one por cada observación

def _predict_one(self, x): → llama a _predict_log_conditional para hacer el argmax de la suma

$$
\log\hat{f}_j(x) + \log\hat{\pi}_j
$$

def _predict_log_conditional(self, x, class_idx)
$$
\log{f_j(x)} = -\frac{1}{2}\log |\Sigma_j| - \frac{1}{2} (x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j) + C
$$
----------------

Buscamos calcular la forma cuadrática para muchas observaciones a la vez:

$$(x-\mu_j)^T \Sigma^{-1} (x- \mu_j)$$

Donde usamos : _predict_log_conditional(x, class_idx)

unbiased_x = x - mean_j

unbiased_x.T @ inv_cov_j @ unbiased_x

## Librerías

In [9]:
# imports
import numpy        as np
import pandas       as pd
import numpy.linalg as LA
from scipy.linalg           import cholesky, solve_triangular
from scipy.linalg.lapack    import dtrtri

from base.qda               import QDA, TensorizedQDA
from base.cholesky          import QDA_Chol1, QDA_Chol2, QDA_Chol3
from utils.bench            import Benchmark
from utils.datasets         import (get_letters_dataset,label_encode)                                                     
from numpy.random           import RandomState

### Dataset 

In [10]:
# dataset de letters
X_letter, y_letter = get_letters_dataset()

# encoding de labels
y_letter_encoded = label_encode(y_letter.reshape(-1,1)) # hago reshape para que quede como matriz columna

# instanciacion del benchmark
b = Benchmark(
    X_letter, y_letter_encoded,
    same_splits=False,
    n_runs=100,
    warmup=20,
    mem_runs=30,
    test_sz=0.2
)

Benching params:
Total runs: 150
Warmup runs: 20
Peak Memory usage runs: 30
Running time runs: 100
Train size rows (approx): 16000
Test size rows (approx): 4000
Test size fraction: 0.2


### Prueba QDA

In [11]:
qda = QDA()

qda.fit(X_letter.T, y_letter_encoded)

qda.predict(X_letter.T[:, :5])

array([[25,  5, 18,  7,  7]])

### Prueba TensorizedQDA

In [12]:
tqda = TensorizedQDA()

tqda.fit(X_letter.T, y_letter_encoded)

tqda.predict(X_letter.T[:, :5])

array([[25,  5, 18,  7,  7]])

### FasterQDA

Definir una clase FasterQDA que herede de TensorizedQDA y redefina predict para predecir todas las observaciones juntas, sin el for sobre filas de X

In [13]:
class FasterQDA(TensorizedQDA):
    """
    Versión vectorizada (sin bucles en predict), 
    con matriz intermedia ineficiente (matriz N x N).
    """

    def _predict_log_conditionals_batch(self, X):
        # Dimensiones iniciales
        k, p, _ = self.tensor_means.shape # self.tensor_means: (k, p, 1)    → k clases, p features
        n = X.shape[1]                    # X: (p, n)                       → p features, n observaciones

        # Cálculo resta: D = X - medias
        D = X[None, :, :] - self.tensor_means   # X: (1, p, n) - Medias: (k, p, 1) → D: (k, p, n)

        # Cálculo de la distancia cuadrática 
        quad_terms = np.empty((k, n)) # Almacena distancias por cada k clase y observación n

        for j in range(k):
            D_j = D[j]                          # Shape: (p, n)
            inv_cov_j = self.tensor_inv_cov[j]  # Shape: (p, p)

            # ---------------------------------------------------------
            # Generación de matriz n x n
            # ---------------------------------------------------------
            # Al multiplicar D_j.T (n, p) @ inv (p, p) @ D_j (p, n), el resultado final es una matriz de (n, n)
            # solo nos importa cada dato consigo mismo (la diagonal)
            
            Q_j = D_j.T @ inv_cov_j @ D_j      # matriz (n, n)

            # Nos quedamos con la diagonal 
            quad_terms[j, :] = np.diag(Q_j)    
            # ---------------------------------------------------------

        
        # log_det es (k,). Lo convertimos a (k, 1) para operar con quad_terms
        log_det = np.log(LA.det(self.tensor_inv_cov))[:, None]
        
        # Fórmula final: 0.5 * log_det - 0.5 * distancia
        return 0.5 * log_det - 0.5 * quad_terms

    def predict(self, X):
        # traigo el log-condicionales 
        log_cond = self._predict_log_conditionals_batch(X)

        # Sumamos el log_a_priori 
        log_post = self.log_a_priori[:, None] + log_cond

        # Elegimos la clase ganadora → por columna
        y_hat = np.argmax(log_post, axis=0)

        return y_hat.reshape(1, -1)

2.5) Demostrar que
$$
diag(A \cdot B) = \sum_{cols} A \odot B^T = np.sum(A \odot B^T, axis=1)
$$ es decir, que se puede "esquivar" la matriz de $n \times n$ usando matrices de $n \times p$. También se puede usar, de forma equivalente,
$$
np.sum(A^T \odot B, axis=0).T
$$

# Demostración paso a paso

### Producto matricial

El elemento $(i,j)$ de $A \cdot B$ se calcula como:
$$
(A \cdot B)_{ij} = \sum_{k=1}^{p} A_{ik} \cdot B_{kj}
$$

Es decir, el **producto interno** entre la fila $i$ de $A$ y la columna $j$ de $B$.

Para la diagonal, solo nos interesan los elementos donde $i = j$:
$$
(A \cdot B)_{ii} = \sum_{k=1}^{p} A_{ik} \cdot B_{ki}
$$

Se observa que $B_{ki}$ es el elemento en la posición $(k, i)$ de $B$, que es exactamente el elemento $(i, k)$ de $B^T$.

Por lo tanto:
$$
(A \cdot B)_{ii} = \sum_{k=1}^{p} A_{ik} \cdot (B^T)_{ik}
$$


Si hacemos el producto elemento a elemento (Hadamard) $A \odot B^T$ :
$$
(A \odot B^T)_{ik} = A_{ik} \cdot (B^T)_{ik}
$$

Y luego sumamos a lo largo de las columnas (axis=1):
$$
\sum_{k=1}^{p} (A \odot B^T)_{ik} = \sum_{k=1}^{p}  A_{ik} \cdot (B^T)_{ik}    = (A \cdot B)_{ii}
$$



### EJEMPLO matriz 2x2

$$
A = \begin{pmatrix}
a & b \\
c & d
\end{pmatrix}
\quad
B = \begin{pmatrix}
e & f \\
g & h
\end{pmatrix}
$$

###  $A \cdot B$

$$
A \cdot B = \begin{pmatrix}
a & b \\
c & d
\end{pmatrix}
\begin{pmatrix}
e & f \\
g & h
\end{pmatrix}
$$

$$
A \cdot B = \begin{pmatrix}
ae + bg & af + bh \\
ce + dg & cf + dh
\end{pmatrix}
$$

###  La diagonal

$$
\text{diag}(A \cdot B) = \begin{pmatrix}
ae + bg \\
cf + dh
\end{pmatrix}
$$

---

###  $B^T$

$$
B^T = \begin{pmatrix}
e & g \\
f & h
\end{pmatrix}
$$

### $A \odot B^T$

$$
A \odot B^T = \begin{pmatrix}
a & b \\
c & d
\end{pmatrix}
\odot
\begin{pmatrix}
e & g \\
f & h
\end{pmatrix}
=
\begin{pmatrix}
a \cdot e & b \cdot g \\
c \cdot f & d \cdot h
\end{pmatrix}
$$

### Suma cada fila (axis=1)

$$
\text{np.sum}(A \odot B^T, \text{axis}=1) = \begin{pmatrix}
ae + bg \\
cf + dh
\end{pmatrix}
$$

---

$$
\text{diag}(A \cdot B) = \begin{pmatrix}
ae + bg \\
cf + dh
\end{pmatrix}
= \text{np.sum}(A \odot B^T, \text{axis}=1)
$$

---



In [17]:
class EfficientQDA(TensorizedQDA):
    """
    Versión que evita la matriz nxn
    usando el producto de Hadamard para calcular solo la diagonal.
    """

    def _predict_log_conditionals_batch(self, X):
        # Dimensiones iniciales
        k, p, _ = self.tensor_means.shape  # self.tensor_means: (k, p, 1) → k clases, p features
        n = X.shape[1]                     # X: (p, n)                    → p features, n observaciones

        # Cálculo resta: D = X - medias
        D = X[None, :, :] - self.tensor_means   # X: (1, p, n) - Medias: (k, p, 1) → D: (k, p, n)

        # Cálculo de la distancia cuadrática 
        quad_terms = np.empty((k, n))  # Almacena distancias por cada k clase y observación n

        for j in range(k):
            D_j = D[j]                          # Shape: (p, n)
            inv_cov_j = self.tensor_inv_cov[j]  # Shape: (p, p)

            # ---------------------------------------------------------
            # MÉTODO EFICIENTE: usando producto de Hadamard
            # ---------------------------------------------------------
            # En vez de hacerr Q_j = D_j.T @ inv_cov_j @ D_j  → (n, n)
            # Calculamos lo siguiente diag(A @ B) = sum(A ⊙ B.T, axis=1)
            
            # 1) A = D_j.T @ inv_cov_j  → (n, p)
            A = D_j.T @ inv_cov_j
            
            # 2) B = D_j  → (p, n), entonces B.T = D_j.T → (n, p)
            # 3) diag(A @ D_j) = sum(A ⊙ D_j.T, axis=1)
            quad_terms[j, :] = np.sum(A * D_j.T, axis=1)
            # ---------------------------------------------------------

        # log_det es (k,). Lo convertimos a (k, 1) para operar con quad_terms
        log_det = np.log(LA.det(self.tensor_inv_cov))[:, None]
        
        # Fórmula final: 0.5 * log_det - 0.5 * distancia
        return 0.5 * log_det - 0.5 * quad_terms

    def predict(self, X):
        # Traigo los log-condicionales 
        log_cond = self._predict_log_conditionals_batch(X)

        # Sumamos el log_a_priori 
        log_post = self.log_a_priori[:, None] + log_cond

        # Elegimos la clase ganadora → por columna
        y_hat = np.argmax(log_post, axis=0)

        return y_hat.reshape(1, -1)

## Comparar modelos con benchmark

In [15]:
# QDA 
b.bench(QDA)

# TensorizedQDA
b.bench(TensorizedQDA)

# FasterQDA
b.bench(FasterQDA)

# EfficientQDA
b.bench(EfficientQDA)

QDA (MEM):   0%|          | 0/30 [00:00<?, ?it/s]

QDA (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

TensorizedQDA (MEM):   0%|          | 0/30 [00:00<?, ?it/s]

TensorizedQDA (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

FasterQDA (MEM):   0%|          | 0/30 [00:00<?, ?it/s]

FasterQDA (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

EfficientQDA (MEM):   0%|          | 0/30 [00:00<?, ?it/s]

EfficientQDA (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

In [16]:
df_summary = b.summary(baseline="QDA")
df_summary

,train_median_ms,train_std_ms,test_median_ms,test_std_ms,mean_accuracy,train_mem_median_mb,train_mem_std_mb,test_mem_median_mb,test_mem_std_mb,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction
model,,,,,,,,,,,,,
QDA,9.6848,1.976466,1734.41890,159.250814,0.886117,0.270096,0.002008,0.098543,0.001085,1.000000,1.000000,1.000000,1.000000
TensorizedQDA,9.1615,1.756816,300.54200,50.063407,0.885303,0.268463,0.002143,0.154099,0.000115,1.057119,5.770970,1.006082,0.639479
FasterQDA,9.8552,3.432659,1921.78170,111.842834,0.884827,0.268951,0.001919,258.234673,0.000689,0.982710,0.902506,1.004255,0.000382
EfficientQDA,7.3710,1.207504,11.51395,1.288257,0.884890,0.269440,0.002171,15.743172,0.000000,1.313906,150.636306,1.002435,0.006259


Vemos que TensorizedQDA entrena en el mismo tiempo que el QDA base y clasifica con la misma accuracy, pero es mucho más rápido en predicción.
FasterQDA no lo mejora porque empieza a comparar todas las observaciones contra todas y arma una matriz gigante N×N que consume 258 MB de RAM y ralentiza todo el proceso.

Por otro lado, se puede vislumbrar que el modelo EfficientQDA mejora a los métodos precesores en términos de tiempo de entrenamiento, como así también se observa una mejora notable en los tiempos de predicción. Además, si bien consume más memoria que los métodos de QDA y TensorizedQDA, tiene una reducción importante comparado a FasterQDA. Estás mejoras en rendimiento general lo hace manteniendo lo mismos niveles de accuracy que logran los demás métodos. 